# Role of Dropout
* paper : https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf


In [27]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [4]:
dataset=pd.read_csv('sonar.all-data',header=None)

In [5]:
print(dataset.head())

       0       1       2       3       4       5       6   ...      54      55      56      57      58      59  60
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  ...  0.0072  0.0167  0.0180  0.0084  0.0090  0.0032   R
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  ...  0.0094  0.0191  0.0140  0.0049  0.0052  0.0044   R
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  ...  0.0180  0.0244  0.0316  0.0164  0.0095  0.0078   R
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  ...  0.0085  0.0073  0.0050  0.0044  0.0040  0.0117   R
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  ...  0.0110  0.0015  0.0072  0.0048  0.0107  0.0094   R

[5 rows x 61 columns]


In [6]:
dataval=dataset.values

In [7]:
print(type(dataval))

<class 'numpy.ndarray'>


In [8]:
# split into input (X) and output (Y) variables
X = dataval[:,0:60].astype(float)
Y = dataval[:,60]

In [9]:
print(len(X))

208


In [10]:
print(len(Y))

208


In [15]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [20]:
def baseline():
    model=Sequential()
    model.add(Dense(60,input_dim=60,activation='relu'))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #compile model
    sgd=SGD(lr=0.01,momentum=0.8)
    model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
    return model

In [21]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Baseline: 87.72% (8.70%)


## Adding dropout at input

In [23]:
def dropout_baseline():
    model=Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60,input_dim=60,activation='relu'))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #compile model
    sgd=SGD(lr=0.01,momentum=0.8)
    model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
    return model

In [24]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=dropout_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 87.38% (7.84%)


## Adding dropout at hidden units

In [25]:
def dropout_baseline():
    model=Sequential()
    
    model.add(Dense(60,input_dim=60,activation='relu'))
    model.add(Dropout(0.2, input_shape=(20,)))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #compile model
    sgd=SGD(lr=0.01,momentum=0.8)
    model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
    return model

In [26]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=dropout_baseline, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Visible: 83.64% (6.14%)
